In [1]:
import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)
 
# 切换到上一层目录
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)
print("切换后的目录：", parent_dir)

当前工作目录： c:\Users\zhaoxs3\Documents\GitHub\NEMESIS\unit_test
切换后的目录： c:\Users\zhaoxs3\Documents\GitHub\NEMESIS


In [2]:
import QuantLib as ql
import numpy as np
import pandas as pd

from devlib.market.curves.overnight_index_curves import Sofr, Estr
from devlib.market.curves.credit_curve_generator import MarketDataCreditCurve, CdsConfig
from devlib.products.credit.cds import Cds

from devlib.utils.ql_date_utils import ql_date_str
from devlib.utils.cds_utils import cds_maturity_date, calendar_5u

In [3]:
today = ql.Date(1,11,2024)
ql.Settings.instance().evaluationDate = today

mkt_file_path = './unit_test/data/ICVS531_curve_data_20241101.xlsx'
swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='swap')
discount_curve = Sofr(today, swap_mkt_data=swap_mkt_data)

entity = 'CHINAGOV'
mkt_file_path = './unit_test/data/credit_curve_data_20241101.xlsx'
cds_type = 'single_name'
cds_data = pd.read_excel(mkt_file_path, sheet_name=cds_type)

cds_config = CdsConfig(recovery_rate = 0.4,
                        daycount = ql.Actual360(),
                        calendar = calendar_5u())
daycount = ql.Actual360()

credit_curve = MarketDataCreditCurve(today, entity, cds_data, discount_curve, cds_type, cds_config, daycount)

In [4]:
from nemesis.products.credit import *
from nemesis.utils import *

####################################################################
#  NEMESIS ALPHA Version 0.1.0 - This build: 24 Jan 2025 at 10:42 #
####################################################################



In [5]:
value_dt = Date(1,11,2024)
issuer_curve = QLCreditCurve(value_dt, credit_curve)

In [6]:
cds_contract = GeneralCDS(
    step_in_dt=value_dt,
    maturity_dt_or_tenor="10Y",
    running_cpn=0.05,
    notional=1e7,
    long_protect=True,
    freq_type=FrequencyTypes.QUARTERLY,
    dc_type=DayCountTypes.ACT_360,
    cal_type=CalendarTypes.CDS,
    bd_type=BusDayAdjustTypes.FOLLOWING,
    dg_type=DateGenRuleTypes.BACKWARD
)

In [7]:
rpv01 = cds_contract.risky_pv01(value_dt, issuer_curve)

In [8]:
rpv01["dirty_rpv01"] * cds_contract.notional * cds_contract.running_cpn

np.float64(4055274.261309052)

In [9]:
cds_contract.value(value_dt, issuer_curve)

{'dirty_pv': np.float64(-3281982.6643738346),
 'clean_pv': np.float64(-3222260.4421516117)}

In [10]:
accrual = cds_contract.running_cpn * cds_contract.notional * (rpv01["dirty_rpv01"] - rpv01["clean_rpv01"])

In [11]:
cds_contract._calc_flows

<bound method GeneralCDS._calc_flows of OBJECT TYPE: GeneralCDS
STEP-IN DATE: 01-NOV-2024
MATURITY: 20-DEC-2034
NOTIONAL: 10000000.0
RUN COUPON: 500.0bp
DAYCOUNT: DayCountTypes.ACT_360
FREQUENCY: FrequencyTypes.QUARTERLY
CALENDAR: CalendarTypes.CDS
BUSDAYRULE: BusDayAdjustTypes.FOLLOWING
DATEGENRULE: DateGenRuleTypes.BACKWARD
ACCRUED DAYS: 43.0
PAYMENT_dt, YEAR_FRAC, ACCRUAL_START, ACCRUAL_END, FLOW
20-DEC-2024,     0.252778, 20-SEP-2024, 19-DEC-2024, 126388.888889
20-MAR-2025,     0.250000, 20-DEC-2024, 19-MAR-2025, 125000.000000
20-JUN-2025,     0.255556, 20-MAR-2025, 19-JUN-2025, 127777.777778
22-SEP-2025,     0.261111, 20-JUN-2025, 21-SEP-2025, 130555.555556
22-DEC-2025,     0.252778, 22-SEP-2025, 21-DEC-2025, 126388.888889
20-MAR-2026,     0.244444, 22-DEC-2025, 19-MAR-2026, 122222.222222
22-JUN-2026,     0.261111, 20-MAR-2026, 21-JUN-2026, 130555.555556
21-SEP-2026,     0.252778, 22-JUN-2026, 20-SEP-2026, 126388.888889
21-DEC-2026,     0.252778, 21-SEP-2026, 20-DEC-2026, 126388.8

In [12]:
prot_pv = cds_contract.prot_leg_pv(value_dt, issuer_curve)
prot_pv

np.float64(773291.5969352175)

In [13]:
cds_contract.credit_dv01(value_dt, issuer_curve, 0.4)

np.float64(10517.777088098228)

In [14]:
cds_contract.interest_dv01(value_dt, issuer_curve, 0.4)

np.float64(1502.1983877439052)